converting widerface annotation files to yolo format annotation file(format change) on train set

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# PATHS (update these if needed)
root_dir = "F:/human behaviour detection/widerface"
image_dir = os.path.join(root_dir, "WIDER_train", "images")
label_dir = os.path.join(root_dir, "WIDER_train", "label")
ann_file = os.path.join(root_dir, "wider_face_split", "wider_face_train_bbx_gt.txt")

def convert_to_yolo(x, y, w, h, img_w, img_h):
    """Convert WIDERFACE bbox to YOLO format"""
    x_center = (x + w/2) / img_w
    y_center = (y + h/2) / img_h
    return x_center, y_center, w/img_w, h/img_h

def process_annotations():
    # Create label directory if not exists
    os.makedirs(label_dir, exist_ok=True)
    
    with open(ann_file, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    i = 0
    stats = {'processed': 0, 'errors': 0, 'missing_images': 0}

    with tqdm(total=len(lines), desc="Converting Annotations") as pbar:
        while i < len(lines):
            # 1. Read image path (e.g. "0--Parade/Parade_1.jpg")
            img_rel_path = lines[i]
            i += 1
            
            # 2. Read number of faces
            try:
                num_faces = int(lines[i])
            except ValueError:
                stats['errors'] += 1
                i += 1
                continue
            i += 1

            # 3. Build paths
            img_path = os.path.join(image_dir, img_rel_path.replace('/', os.sep))
            label_path = os.path.join(label_dir, f"{os.path.splitext(img_rel_path)[0]}.txt")
            os.makedirs(os.path.dirname(label_path), exist_ok=True)

            # 4. Skip if image missing
            if not os.path.exists(img_path):
                stats['missing_images'] += 1
                i += num_faces  # Skip bbox lines
                continue

            # 5. Get image dimensions
            try:
                with Image.open(img_path) as img:
                    img_w, img_h = img.size
            except Exception as e:
                stats['errors'] += 1
                i += num_faces
                continue

            # 6. Process bounding boxes
            with open(label_path, 'w') as lbl_file:
                for _ in range(num_faces):
                    if i >= len(lines):
                        break
                    
                    # Parse bbox (x,y,w,h)
                    try:
                        x, y, w, h = map(float, lines[i].split()[:4])
                    except:
                        i += 1
                        continue
                    
                    # Validate bbox
                    if w <= 0 or h <= 0:
                        i += 1
                        continue
                    
                    # Convert to YOLO format
                    x = max(0, min(x, img_w-1))
                    y = max(0, min(y, img_h-1))
                    xc, yc, wn, hn = convert_to_yolo(x, y, w, h, img_w, img_h)
                    
                    # Write to file (class 0 for face)
                    lbl_file.write(f"0 {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}\n")
                    i += 1

            stats['processed'] += 1
            pbar.update(2 + num_faces)  # Update for image + num_faces lines

    # Print summary
    print("\n=== Conversion Summary ===")
    print(f"Processed images: {stats['processed']}")
    print(f"Missing images: {stats['missing_images']}")
    print(f"Errors: {stats['errors']}")
    print(f"Label files saved to: {label_dir}")

if __name__ == "__main__":
    process_annotations()

Annotated images(images with bounding boxes being put on main object(i.e face)) for checking whether the files has been converted properly or not, but on train set

In [ ]:
import os
import cv2

def draw_boxes_on_images(image_root, label_root, output_root):
    for root, _, files in os.walk(image_root):
        for file in files:
            if not file.lower().endswith(".jpg"):
                continue

            image_path = os.path.join(root, file)
            rel_path = os.path.relpath(image_path, image_root)
            image_name_wo_ext = os.path.splitext(file)[0]

            label_path = os.path.join(label_root, os.path.dirname(rel_path), image_name_wo_ext + ".txt")
            if not os.path.exists(label_path):
                print(f"[Warning] Missing annotation for: {image_path}")
                continue

            img = cv2.imread(image_path)
            if img is None:
                print(f"[Error] Cannot read image: {image_path}")
                continue

            h, w = img.shape[:2]

            with open(label_path, "r") as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"[Error] Invalid line in {label_path}: {line.strip()}")
                    continue

                cls_id, x_center, y_center, width, height = map(float, parts)
                if any(v < 0 or v > 1 for v in [x_center, y_center, width, height]):
                    print(f"[Warning] Out-of-range coords in {label_path}: {line.strip()}")
                    continue

                # Convert YOLO format to pixel format
                x1 = int((x_center - width / 2) * w)
                y1 = int((y_center - height / 2) * h)
                x2 = int((x_center + width / 2) * w)
                y2 = int((y_center + height / 2) * h)

                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, str(int(cls_id)), (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

            # Prepare the output path
            output_path = os.path.join(output_root, rel_path)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            cv2.imwrite(output_path, img)
            print(f"Saved: {output_path}")


# === Run the function ===

image_dir = r"F:\human behaviour detection\widerface\WIDER_train\images"
label_dir = r"F:\human behaviour detection\widerface\WIDER_train\labels"
output_dir = r"F:\human behaviour detection\widerface\WIDER_train\annotated"

draw_boxes_on_images(image_dir, label_dir, output_dir)


anchor boxes generation using k means

In [ ]:
import os
import numpy as np
from sklearn.cluster import KMeans

def load_yolo_labels(label_dir, image_size=416):
    box_dims = []
    for root, _, files in os.walk(label_dir):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r') as f:
                    for line in f.readlines():
                        parts = line.strip().split()
                        if len(parts) != 5:
                            continue
                        _, x_center, y_center, width, height = map(float, parts)
                        box_dims.append([width * image_size, height * image_size])
    return np.array(box_dims)

def iou(box, clusters):
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]
    union = box_area + cluster_area - intersection
    return intersection / union

def avg_iou(boxes, clusters):
    return np.mean([np.max(iou(box, clusters)) for box in boxes])

def kmeans(boxes, k, dist=np.median, seed=1):
    np.random.seed(seed)
    indices = np.random.choice(boxes.shape[0], k, replace=False)
    clusters = boxes[indices]
    last_clusters = np.zeros((boxes.shape[0],))
    
    while True:
        distances = []
        for box in boxes:
            distances.append(1 - iou(box, clusters))
        distances = np.array(distances)
        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for i in range(k):
            clusters[i] = dist(boxes[nearest_clusters == i], axis=0)
        last_clusters = nearest_clusters

    return clusters

# ---- SET PATH TO YOUR LABEL DIRECTORY ----
label_dir = "F:/human_behaviour_detection/widerface/WIDER_train/labels"
image_size = 416
num_anchors = 5

# Load bounding boxes
boxes = load_yolo_labels(label_dir, image_size=image_size)

# Run k-means clustering
anchors = kmeans(boxes, k=num_anchors)

# Sort anchors by area
anchors = anchors[np.argsort(anchors[:, 0] * anchors[:, 1])]

# Normalize (optional)
anchors_normalized = anchors / image_size
print("Normalized anchors (for YOLO config):")
print(anchors_normalized)

print("Average IoU:", avg_iou(boxes, anchors))


code for converting the labels format(widerface format ) to yolo format for validation set

In [ ]:
import os
from PIL import Image

# Base paths (customize these for your setup)
val_images_dir = r"F:\human behaviour detection\widerface\WIDER_val\images"
val_labels_dir = r"F:\human behaviour detection\widerface\WIDER_val\labels"
annotation_file = r"F:\human behaviour detection\widerface\wider_face_split\wider_face_val_bbx_gt.txt"

# Make sure output label folders exist
os.makedirs(val_labels_dir, exist_ok=True)

def convert_to_yolo(x, y, w, h, img_w, img_h):
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    return [0, x_center, y_center, w / img_w, h / img_h]  # class_id = 0

with open(annotation_file, 'r') as f:
    lines = f.readlines()

i = 0
while i < len(lines):
    img_rel_path = lines[i].strip()
    i += 1
    num_boxes = int(lines[i].strip())
    i += 1

    # Resolve full image path
    img_path = os.path.join(val_images_dir, img_rel_path)
    label_path = os.path.join(val_labels_dir, img_rel_path.replace(".jpg", ".txt"))

    os.makedirs(os.path.dirname(label_path), exist_ok=True)

    try:
        img = Image.open(img_path)
        img_w, img_h = img.size
    except FileNotFoundError:
        print(f"❌ Image not found: {img_path}")
        i += num_boxes
        continue

    with open(label_path, 'w') as out_f:
        for _ in range(num_boxes):
            parts = lines[i].strip().split()
            i += 1
            if len(parts) < 4:
                continue

            x, y, w, h = map(int, parts[:4])
            if w <= 0 or h <= 0:
                continue  # skip invalid boxes

            yolo_box = convert_to_yolo(x, y, w, h, img_w, img_h)
            out_f.write(" ".join(f"{val:.6f}" for val in yolo_box) + "\n")

print("✅ All validation annotations successfully converted to YOLO format.")
print(f"Saved under: {val_labels_dir}")


code to verify the whether converted format obatined are correctly detecting the object in images so craetion of folder name annoated whre the imaages has bouding box based on its correspodning label file for validation datasets

In [ ]:
import os
import cv2

# Directories
base_dir = r"F:\human behaviour detection\widerface\WIDER_val"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")
annotated_dir = os.path.join(base_dir, "annotated")

os.makedirs(annotated_dir, exist_ok=True)

def draw_boxes(img_path, label_path, save_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ Could not load: {img_path}")
        return

    h, w, _ = img.shape

    try:
        with open(label_path, 'r') as f:
            for line in f:
                parts = list(map(float, line.strip().split()))
                if len(parts) != 5:
                    continue
                _, x_center, y_center, width, height = parts

                # Convert back to pixel coordinates
                x1 = int((x_center - width / 2) * w)
                y1 = int((y_center - height / 2) * h)
                x2 = int((x_center + width / 2) * w)
                y2 = int((y_center + height / 2) * h)

                # Draw rectangle and label
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, 'face', (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    except FileNotFoundError:
        print(f"⚠️ No label for: {img_path}")
        return

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    cv2.imwrite(save_path, img)

# Walk through all image files and visualize annotations
for root, _, files in os.walk(images_dir):
    for file in files:
        if file.endswith(".jpg"):
            rel_path = os.path.relpath(os.path.join(root, file), images_dir)
            img_path = os.path.join(images_dir, rel_path)
            label_path = os.path.join(labels_dir, rel_path.replace(".jpg", ".txt"))
            save_path = os.path.join(annotated_dir, rel_path)

            draw_boxes(img_path, label_path, save_path)

print("✅ Annotated images saved to:", annotated_dir)
